In [ ]:
!pip install sacremoses
!pip install av
!pip install transformers

In [ ]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer, AutoImageProcessor, AutoTokenizer, VisionEncoderDecoderModel
import av
import numpy as np
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

#translation
mname = "facebook/wmt19-en-ru"
tr_tokenizer = FSMTTokenizer.from_pretrained(mname)
translate = FSMTForConditionalGeneration.from_pretrained(mname).to(device)

# load pretrained processor, tokenizer, and model for video processing
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = VisionEncoderDecoderModel.from_pretrained("Neleac/timesformer-gpt2-video-captioning").to(device)

In [ ]:
def VideoEncode(image_processor, tokenizer, model, video_path, tr_tokenizer, translate)

    try:
        container = av.open(video_path)
    except:
        continue

  # extract evenly spaced frames from video
    captions = []
    total_len = container.streams.video[0].frames
    clip_len = model.config.encoder.num_frames
    for i in range((total_len//30)//clip_len):
        # print(i)
        fp = i*30*8
        sp = fp+240
        indices = set(np.linspace(fp, sp, num=clip_len, endpoint=False).astype(np.int64))
        # print(indices)
        frames = []
        container.seek(0)
        for k, frame in enumerate(container.decode(video=0)):
            if k in indices:
                frames.append(frame.to_ndarray(format="rgb24"))
                # generate caption
        gen_kwargs = {
        "min_length": 20,
        "max_length": 32,
        "num_beams": 8,}
        pixel_values = image_processor(frames, return_tensors="pt").pixel_values.to(device)
        tokens = model.generate(pixel_values, **gen_kwargs)
        caption = tokenizer.batch_decode(tokens, skip_special_tokens=True)[0]
        captions.append(caption)
        video_inf = ' '.join(captions)
        
        #translation
        input = video_inf
        input_ids = tr_tokenizer.encode(input, return_tensors="pt").to(device)
        outputs = translate.generate(input_ids)
        decoded = tr_tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        return decoded